In [108]:
import tqdm
import random
import pathlib
import itertools
import collections

import pandas as pd
import os
import cv2
import numpy as np
import remotezip as rz

import tensorflow as tf

# Some modules to display an animation using imageio.
import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed

In [109]:
df = pd.read_csv(os.path.join(csv_dir, csv_file))

In [110]:
# Create a directory to store all the clips
clips_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\All_Clips"
os.makedirs(clips_dir, exist_ok=True)

# Loop through all the CSV files
csv_dir = r'C:\Users\BEE Team\Desktop\code\Summarys_labels'
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
csv_files = csv_files[1:]

for csv_file in csv_files:
    if csv_file.endswith('.csv'):
        # Read the CSV file
        df = pd.read_csv(os.path.join(csv_dir, csv_file))

        # Directory of videos
        video_dir = r'C:\Users\BEE Team\Desktop\Videos\Daniele_Vids_Lab\LD\Yellow'
        df['Video_path'] = df['file'].apply(lambda x: os.path.join(video_dir, x.split('_labels')[0] + '.mp4'))

        # Create a directory to store the clips for this csv file
        csv_clips_dir = clips_dir + "\clips_" + os.path.splitext(csv_file.split('_')[2])[0]
        os.makedirs(csv_clips_dir, exist_ok=True)

        # Extract video clips from each row of the dataframe and save them to a file
        for index, row in df.iterrows():
            video_path = row['Video_path']
            start_frame = row['start_frame']
            end_frame = row['end_frame']
            classification = row['classification']
            
            video = cv2.VideoCapture(video_path)
            video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            clip_frames = []
            for i in range(start_frame, end_frame):
                ret, frame = video.read()
                if ret:
                    clip_frames.append(frame)
                else:
                    break
            
            video.release()
            
            clip_path = os.path.join(csv_clips_dir, f'{classification}_{index}.mp4')
            clip = cv2.VideoWriter(clip_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (frame.shape[1], frame.shape[0]))
            for frame in clip_frames:
                clip.write(frame)
            clip.release()
